# Slope Comparisons

In [18]:
import pandas as pd
import numpy as np
import altair as alt
from scipy import stats
from altair import datum
from ipywidgets import interact

# Set up some global config and variables
alt.renderers.enable('default')
pd.options.mode.chained_assignment = None
np.seterr(all='ignore')

df = pd.read_csv('jhu-daily-reports.csv')
df['Active']  = df.Confirmed - (df.Deaths + df.Recovered)
samples = df[['Date', 'Country']].groupby('Date').Country.nunique()
days = samples[samples > 1].index.tolist()
df = df[df['Date'].isin(days)]

country_level = df.groupby(['Country', 'Date'], as_index=False).sum()
def state_data(country):
    return df[df['Country'] == country].groupby(['State', 'Date'], as_index=False).sum()
def county_data(state):
    return df[(df['Country'] == 'US') & (df['State'] == state)].groupby(['County', 'Date'], as_index=False).sum()


In [19]:
def slope_chart(data, by, offset, xscale='linear', limit=400, scale=1, value='Confirmed_New', window=7, today=days[-1]):
    data = data[data['Date']<=today]
    source = data[data['Date'] == today]
    for var in source[by].unique():
        values = data[data[by] == var].sort_values('Date').tail(window)[['Confirmed', value]]
        slope, intercept, r_value, p_value, std_err = stats.linregress(values.Confirmed, values[value])
        source.loc[source[by] == var, 'Slope'] = slope
    source.fillna(0, inplace=True)
    source = source[source[value] > limit]

    title = 'Slope of %s in last %d days since %s vs. Total Confirmed' % (' '.join(value.split('_')), window, today)
    base = alt.Chart(source, title=title).mark_point(filled=True, stroke='grey').encode(
        alt.X('Confirmed:Q', scale=alt.Scale(type=xscale), axis=alt.Axis(offset=offset)),
        y='Slope:Q',
        color=alt.Color(by+':N', scale=alt.Scale(scheme='category20'), legend=alt.Legend(columns=2, clipHeight=20, padding=10)),
        size=alt.Size(value+':Q', scale=alt.Scale(domain=[source.Confirmed_New.min(), source.Confirmed_New.max()], range=[100*scale, 3000*scale])),
        tooltip=[by, 'Confirmed', 'Slope', value]
    )
    text = base.mark_text().encode(
        text=by+':N',
        size=alt.value(12),
        color=alt.value('black')
    ).transform_filter(datum[value] > limit*2)
    regression = base.transform_regression('Confirmed', 'Slope', method="poly", order=1).mark_line(strokeDash=[6,8]).encode(color=alt.value('grey'), size=alt.value(2))
    hline = alt.Chart().mark_rule(color='red', strokeDash=[6,3]).encode(y='a:Q').transform_calculate(a="0")

    return (base+text+regression+hline) if offset == 0 else (base+text+regression)

# Country Level

In [20]:
slope_chart(country_level, 'Country', -200, xscale='log', limit=500, scale=5, window=7).properties(
    width=1200,
    height=800
).interactive()

alt.LayerChart(...)

In [21]:
data = country_level
state = alt.Chart(data[data['Country'] == 'US'].sort_values('Date').tail(30)).mark_line().encode(
    alt.X('Confirmed:Q', axis=alt.Axis(title='Cumulative Cases')),
    alt.Y('Confirmed_New:Q', axis=alt.Axis(title='New Cases'))
)
reg = state.transform_regression("Confirmed", "Confirmed_New", method="poly").mark_line(color='red', strokeDash=[6,3])
(state+reg).interactive()

alt.LayerChart(...)

# State Level

In [22]:
@interact(window=(2, 21, 1))
def chart(window=7):
    return slope_chart(state_data('US'), 'State', 0, limit=200, xscale='log', scale=2, window=window).properties(
        width=1200,
        height=800
    ).interactive()

interactive(children=(IntSlider(value=7, description='window', max=21, min=2), Output()), _dom_classes=('widge…

In [23]:
data = state_data('US')
state = alt.Chart(data[data['State'] == 'SC'].sort_values('Date').tail(60)).mark_line().encode(
    x='Confirmed:Q',
    y='Confirmed_New:Q'
)
reg = state.transform_regression("Confirmed", "Confirmed_New", method="poly").mark_line(color='red', strokeDash=[6,3])
(state+reg).interactive()

alt.LayerChart(...)

# US County Level

In [24]:
@interact(window=(2, 21, 1))
def chart(window=7):
    return slope_chart(county_data('FL'), 'County', 0, xscale='log', limit=1, scale=7, window=window).properties(
        width=1200,
        height=800
    ).interactive()

interactive(children=(IntSlider(value=7, description='window', max=21, min=2), Output()), _dom_classes=('widge…

In [25]:
data = county_data('CA')
state = alt.Chart(data[data['County'] == 'Los Angeles'].sort_values('Date').tail(60)).mark_line().encode(
    x='Confirmed:Q',
    y='Confirmed_New:Q'
)
reg = state.transform_regression("Confirmed", "Confirmed_New", method="poly").mark_line(color='red', strokeDash=[6,3])
(state+reg).interactive()

alt.LayerChart(...)

# US Hospitalizations

In [32]:
dfh = pd.read_csv('https://covidtracking.com/api/v1/states/daily.csv')
dfh.date = pd.to_datetime(dfh.date, format='%Y%m%d')
dfh.date = dfh.date.dt.strftime('%m-%d-%Y')
dfh = dfh.rename({'date': 'Date', 'state':'State', 'hospitalizedCurrently': 'Hospitalized'}, axis=1)
data = state_data('US')
data = data.merge(dfh, on=['Date', 'State'], how='outer')
@interact(window=(2, 21, 1))
def chart(window=7):
    return slope_chart(data, 'State', -492, xscale='log', limit=200, scale=2, value='Hospitalized', window=window, today=days[-1]).properties(
        width=1200,
        height=800
    ).interactive()

interactive(children=(IntSlider(value=7, description='window', max=21, min=2), Output()), _dom_classes=('widge…

In [27]:
state = alt.Chart(data[data['State'] == 'TX'].sort_values('Date').tail(21)).mark_line().encode(
    x='Confirmed:Q',
    y='Hospitalized:Q'
)
reg = state.transform_regression("Confirmed", "Hospitalized", method="poly").mark_line(color='red', strokeDash=[6,3])
(state+reg).interactive()

alt.LayerChart(...)